In [ ]:
import requests
import pyttsx3
import matplotlib.pyplot as plt
import os

# API 設定
CITY_API = 'https://opendata.cwa.gov.tw/api/v1/rest/datastore/F-C0032-001'
DISTRICT_API = 'https://opendata.cwa.gov.tw/api/v1/rest/datastore/F-D0047-093'
API_KEY = 'CWA-F672F228-AA9D-4184-8976-269CD9D19B0C'

# 使用者輸入
LOCATION = input("請輸入市級地區：").strip()
TIME_FILTER = input("請選擇時段（白天 / 夜間 / 全部）：").strip()

# 初始化語音
engine = pyttsx3.init()
engine.setProperty('rate', 1500)
engine.setProperty('volume', 0.9)

def is_district(location):
    return location.endswith("區") or location.endswith("鄉") or location.endswith("鎮") or location in ["新竹市", "嘉義市"]

def fetch_city_data(location):
    res = requests.get(CITY_API, params={'Authorization': API_KEY, 'locationName': location})
    data = res.json()
    location_data = data['records']['location'][0]
    name = location_data['locationName']
    elements = {e['elementName']: e['time'] for e in location_data['weatherElement']}
    count = min(len(elements['Wx']), len(elements['PoP']), len(elements['MinT']), len(elements['MaxT']), 4)
    results = []
    for i in range(count):
        start = elements['Wx'][i]['startTime']
        hour = int(start[11:13])
        period = "白天" if hour == 6 else "夜間"
        if TIME_FILTER != "全部" and TIME_FILTER != period:
            continue
        weather = elements['Wx'][i]['parameter']['parameterName']
        rain = int(elements['PoP'][i]['parameter']['parameterName'])
        tmin = int(elements['MinT'][i]['parameter']['parameterName'])
        tmax = int(elements['MaxT'][i]['parameter']['parameterName'])
        avg = round((tmin + tmax) / 2)
        results.append({
            "時間": f"{elements['Wx'][i]['startTime']} ~ {elements['Wx'][i]['endTime']}（{period}）",
            "天氣": weather,
            "降雨機率": f"{rain}%",
            "溫度": f"{avg}°C",
            "建議": generate_advice(weather, rain, avg)
        })
    return name, results

def fetch_district_data(location):
    res = requests.get(DISTRICT_API, params={
        'Authorization': API_KEY,
        'locationName': location,
        'elementName': 'WeatherDescription,PoP12h,T'
    })
    data = res.json()
    location_data = data['records']['locations'][0]['location'][0]
    name = location_data['locationName']
    elements = {e['elementName']: e['time'] for e in location_data['weatherElement']}
    count = min(len(elements['WeatherDescription']), len(elements['PoP12h']), len(elements['T']), 8)
    results = []
    for i in range(count):
        start = elements['WeatherDescription'][i]['startTime']
        hour = int(start[11:13])
        if 5 <= hour < 11:
            period = "早上"
        elif 11 <= hour < 17:
            period = "中午"
        else:
            period = "晚上"
        if TIME_FILTER != "全部" and TIME_FILTER != period:
            continue
        weather = elements['WeatherDescription'][i]['elementValue'][0]['value']
        rain = int(elements['PoP12h'][i]['elementValue'][0]['value'])
        temp = int(elements['T'][i]['elementValue'][0]['value'])
        results.append({
            "時間": f"{start} ~ {elements['WeatherDescription'][i]['endTime']}（{period}）",
            "天氣": weather,
            "降雨機率": f"{rain}%",
            "溫度": f"{temp}°C",
            "建議": generate_advice(weather, rain, temp)
        })
    return name, results

def generate_advice(weather, rain_prob, temp):
    advice = []
    if "雨" in weather or rain_prob >= 50:
        advice.append("記得帶傘")
    if temp >= 30:
        advice.append("防曬與補水")
    if temp <= 18:
        advice.append("注意保暖")
    if not advice:
        advice.append("天氣穩定，適合外出")
    return "｜".join(advice)

def generate_chart(forecast):
    """生成天氣預報圖表並儲存到桌面"""
    import matplotlib
    matplotlib.rcParams['font.sans-serif'] = ['Microsoft JhengHei']  # 解決中文亂碼
    matplotlib.rcParams['axes.unicode_minus'] = False

    if not forecast:
        print("無預報資料，無法生成圖表")
        return

    # 精簡時間格式
    times = [f["時間"].split(" ")[0] + f["時間"].split("（")[-1].replace("）", "") for f in forecast]
    temps = [float(f["溫度"].replace("°C", "")) for f in forecast]
    rain_probs = [int(f["降雨機率"].replace("%", "")) for f in forecast]

    # 繪圖
    plt.figure(figsize=(10, 6))
    plt.plot(times, temps, 'b-o', label='平均溫度 (°C)', linewidth=2, markersize=6)
    plt.bar(times, rain_probs, color='lightblue', alpha=0.6, label='降雨機率 (%)')

    # 加數字標籤
    for i, (t, r) in enumerate(zip(times, rain_probs)):
        plt.text(i, r + 1, f"{r}%", ha='center', fontsize=9)
    for i, (t, temp) in enumerate(zip(times, temps)):
        plt.text(i, temp + 0.5, f"{temp}°", ha='center', fontsize=9, color='blue')

    # 格式化與儲存
    plt.title(f'{LOCATION} 未來天氣預報')
    plt.xlabel('時間')
    plt.ylabel('溫度 / 降雨機率')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop", "weather_forecast.png")
    plt.savefig(desktop_path)
    plt.close()
    print("✅ 圖表已儲存到桌面：", desktop_path)



def speak_forecast(loc_name, forecast):
    text = f"{loc_name} 的 {TIME_FILTER} 天氣預報如下：\n"
    for f in forecast:
        text += f"{f['時間']}，{f['天氣']}，降雨機率 {f['降雨機率']}，溫度 {f['溫度']}，建議：{f['建議']}。\n"
    engine.say(text)
    engine.runAndWait()

# 主流程
try:
    if is_district(LOCATION):
        loc, forecast = fetch_district_data(LOCATION)
    else:
        loc, forecast = fetch_city_data(LOCATION)

    if forecast:
        for f in forecast:
            print("—" * 40)
            for k, v in f.items():
                print(f"{k}：{v}")
        speak_forecast(loc, forecast)
        generate_chart(forecast)
        print("已生成天氣預報圖表：weather_forecast.png")
        print("圖表儲存在：", os.path.abspath("weather_forecast.png"))
    else:
        print("⚠ 找不到符合時段的預報資料")
except Exception as e:
    print("⚠ 錯誤：", e)

print("""
資料來源：交通部中央氣象署
https://www.cwa.gov.tw
""")
